# URDF experiments for design on demand

URDF units are written and joined with the help of the ODIO URDF library and constructed robots are played around with using the teleop from pydrake

In [ ]:
# Imports
import numpy as np
import altair as alt
import pydot
from IPython.display import display, SVG, clear_output
import plotly.express as px

from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, InverseKinematics,
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MeshcatVisualizerCpp, MultibodyPlant, MultibodyPositionToGeometryPose, Parser,
    PiecewisePose, PiecewisePolynomial, Quaternion, RigidTransform, Solve,
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource
)
from pydrake.examples.manipulation_station import ManipulationStation

from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import (
    StartMeshcat, MeshcatJointSlidersThatPublish, MeshcatPoseSliders)
from manipulation.scenarios import AddMultibodyTriad, SetColor

import pandas as pd

In [ ]:
from odio_urdf import *
import math
import sys
# from operator import add
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
# from design-on-demand-21-12-14 import *

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
# Joint teleop
def gripper_forward_kinematics_example():
    builder = DiagramBuilder()
   
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile('./test.urdf', 'lcl-DoD')
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame())
    plant.Finalize()

    # Draw the frames
    for body_name in ["root_mod", "linkMod_8"]:
        AddMultibodyTriad(plant.GetFrameByName(body_name, model), scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

#     plant.SetPositions(plant.GetMyContextFromRoot(context),
#                   plant.GetModelInstanceByName("diva_robot"),
#                   [-3.54, -2.33, -1.49, -1.62, -0.97, -0.19, 0.47])
    
    gripper = plant.GetBodyByName("linkMod_8")
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        print(pose.translation())
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print("pose rotation: ", pose.rotation())
    sliders = MeshcatJointSlidersThatPublish(meshcat, plant, visualizer, context)
    # sliders.Run()
    sliders.Run(pose_callback)

gripper_forward_kinematics_example()